In [1]:
!git clone https://github.com/roihezkiyahu/CWGAN-GP.git

Cloning into 'CWGAN-GP'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 29 (delta 10), reused 12 (delta 6), pack-reused 0
Unpacking objects: 100% (29/29), 4.69 MiB | 8.86 MiB/s, done.


In [2]:
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.3 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: starlette
    Found existing installation: starlette 0.26.1
    Uninstalling starlette-0.26.1:
      Successfully uninstalled starlette-0.26.1
  Attempting uninstall: fastapi
    Found existing installation: fastapi 0.95.1
    Uninstalling fastapi-0.95.1:
      Successfully uninstalled fastapi-0.95.1


In [3]:
%cd CWGAN-GP

/kaggle/working/CWGAN-GP


In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
from Generator import GeneratorMNIST
from Discriminator import DiscriminatorMNIST
from GAN import GAN
import lightning as pl
from PIL import Image
from IPython.display import Image as DisplayImage, display
import os
import re
import numpy as np
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [5]:
batch_size = 64
learning_rate_wgan = 2e-4
num_epochs = 150
lambda_gp = 10
latent_dim = 128
dim = 128
n_critics = 1
step_lr_gamma = 0.975
betas = (0.5, 0.999)
acc_device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(acc_device)

In [6]:
# image processing
transform = transforms.Compose([
    transforms.Pad(2),
    transforms.ToTensor(),
    transforms.RandomAffine(5, (0.1, 0.1), scale=(0.95, 1.05)),
    transforms.Normalize([0.5], [0.5]),
])

# Configuring Dataset
train_set = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

full_dataset = torch.utils.data.ConcatDataset([train_set, test_set])

train_loader = torch.utils.data.DataLoader(full_dataset, batch_size=batch_size,
                                           shuffle=True, num_workers=2, pin_memory=True)
class_names = train_set.classes

100%|██████████| 26421880/26421880 [00:01<00:00, 17402100.60it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 307720.89it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5660504.76it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 21357346.18it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [7]:
# Configurations and other code from your script...

# CWGAN-GP
generator_wgan_mnist = GeneratorMNIST((32, 32, 1), latent_dim, dim, batch_norm=True)
discriminator_wgan_mnist = DiscriminatorMNIST((32, 32, 1), dim, batch_norm=False)
wgan_mnist = GAN(generator_wgan_mnist, discriminator_wgan_mnist, True, lambda_gp, device, learning_rate_wgan,
                 betas, n_critics, step_lr_gamma=step_lr_gamma, class_names=class_names)


# Training
trainer = pl.Trainer(accelerator=acc_device, max_epochs=num_epochs)
trainer.fit(wgan_mnist, train_loader)


INFO: GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO: 
  | Name               | Type               | Params
----------------------------------------------------------
0 | generator          | GeneratorMNIST     | 3.8 M 
1 | discriminator      | DiscriminatorMNIST | 1.7 M 
2 | generator_loss     | BCELoss            | 0     
3 | discriminator_loss | BCELoss            | 0     
----------------------------------------------------------
5.5 M     Trainable params
0         Non-trainable params
5.5 M     Total params
22.054    Total estimated model params size (MB)
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on t

Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
